### Auto-Labeling dataset preparation

### FIXME

1. Choose between default and custom dataset in FIXME 1
1. Assign path of DATA_DIR in FIXME 2

In [ ]:
model_name = "mal"

### Example dataset source and structure <a class="anchor" id="head-1.1"></a>

We will be using the `COCO dataset`. `download_coco.sh` script from dataset prepare will be used to download and unzip the coco2017 dataset from [here](https://cocodataset.org/#download)


**If using custom dataset; it should follow this dataset structure**
```
DATA_DIR
├── annotations.json
├── images
    ├── image_name_1.jpg
    ├── image_name_2.jpg
    ├── ...

```

In [ ]:
import os

dataset_to_be_used = "default" #FIXME1 #default/custom; default for the dataset used in this tutorial notebook; custom for a different dataset
DATA_DIR = model_name #FIXME2
os.environ['DATA_DIR']= DATA_DIR
!mkdir -p $DATA_DIR

### Download dataset

In [ ]:
if dataset_to_be_used == "default":
    !bash coco/download_coco.sh $DATA_DIR
    # Remove existing data
    !rm -rf $DATA_DIR/train2017/images
    !rm -rf $DATA_DIR/val2017/images
    # Rearrange data in the required format
    !mkdir -p $DATA_DIR/train2017/
    !mkdir -p $DATA_DIR/val2017/
    !mv $DATA_DIR/raw-data/train2017 $DATA_DIR/train2017/images
    !mv $DATA_DIR/raw-data/annotations/instances_train2017.json $DATA_DIR/train2017/annotations.json
    !mv $DATA_DIR/raw-data/val2017 $DATA_DIR/val2017/images
    !mv $DATA_DIR/raw-data/annotations/instances_val2017.json $DATA_DIR/val2017/annotations.json


### Verify the downloaded dataset

In [ ]:
!if [ ! -d $DATA_DIR/train2017/images ]; then echo 'Images folder not found'; else echo 'Found images folder';fi
!if [ ! -f $DATA_DIR/train2017/annotations.json ]; then echo 'annotations file not found'; else echo 'Found annotations file';fi
!if [ ! -d $DATA_DIR/val2017/images ]; then echo 'Images folder not found'; else echo 'Found images folder';fi
!if [ ! -f $DATA_DIR/val2017/annotations.json ]; then echo 'annotations file not found'; else echo 'Found annotations file';fi

In [ ]:
# extract a subset of images from COCO dataset
# comment out if you need full dataset
!python3 mask_grounding_dino/extract_subset.py $DATA_DIR/train2017/images $DATA_DIR/train2017/annotations.json $DATA_DIR/train2017_subset 100
!python3 mask_grounding_dino/extract_subset.py $DATA_DIR/val2017/images $DATA_DIR/val2017/annotations.json $DATA_DIR/val2017_subset true 5
assert (os.path.exists(f"{DATA_DIR}/train2017_subset/images"))
assert (os.path.exists(f"{DATA_DIR}/train2017_subset/annotations.json"))
assert (os.path.exists(f"{DATA_DIR}/val2017_subset/images"))
assert (os.path.exists(f"{DATA_DIR}/val2017_subset/annotations.json"))

In [ ]:
!mkdir -p $DATA_DIR/cloud_folders/data/train
!mkdir -p $DATA_DIR/cloud_folders/data/val

!tar -C $DATA_DIR/train2017_subset -czf $DATA_DIR/cloud_folders/data/train/images.tar.gz images
!tar -C $DATA_DIR/val2017_subset -czf $DATA_DIR/cloud_folders/data/val/images.tar.gz images
!cp $DATA_DIR/train2017_subset/annotations.json $DATA_DIR/cloud_folders/data/train/annotations.json
!cp $DATA_DIR/val2017_subset/annotations.json $DATA_DIR/cloud_folders/data/val/annotations.json
!cp coco_panoptic/labelmap.json $DATA_DIR/cloud_folders/data/train/label_map.json
!cp coco_panoptic/labelmap.json $DATA_DIR/cloud_folders/data/val/label_map.json

### Final step: Upload the tar files to your cloud storage and move on to running the API requests example notebooks

In [ ]:
!python3 -m pip install --upgrade awscli
ACCESS_KEY=FIXME4.1
SECRET_KEY=FIXME4.2
BUCKET_NAME=FIXME4.3

!AWS_ACCESS_KEY_ID={ACCESS_KEY} AWS_SECRET_ACCESS_KEY={SECRET_KEY} aws s3 cp $DATA_DIR/cloud_folders/data/train s3://{BUCKET_NAME}/data/auto_label_train --recursive
!AWS_ACCESS_KEY_ID={ACCESS_KEY} AWS_SECRET_ACCESS_KEY={SECRET_KEY} aws s3 cp $DATA_DIR/cloud_folders/data/val s3://{BUCKET_NAME}/data/auto_label_val --recursive

In [ ]:
# This will be the paths in your API/TAO-CLIENT Notebooks
train_dataset_path = f"/data/auto_label_train"
eval_dataset_path = f"/data/auto_label_val"